<a href="https://colab.research.google.com/github/yunyun3599/MLOps/blob/main/wandb_sweeps_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -q wandb

     |████████████████████████████████| 1.7 MB 5.2 MB/s 
     |████████████████████████████████| 144 kB 47.1 MB/s 
     |████████████████████████████████| 180 kB 55.6 MB/s 
     |████████████████████████████████| 63 kB 1.2 MB/s 


In [2]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [13]:
# %load train_lib.py

def train():
    import numpy as np
    import tensorflow as tf
    import wandb
    config_defaults = {
        'layer1_size': 128,
        'layer1_activation':'relu',
        'dropout_rate':0.2,
        'optimizer':'adam',
        'learning_rate':0.01
    }
    wandb.init(project='sweep-practice', config=config_defaults, magic=True)
    config = wandb.config

    fashion_mnist = tf.keras.datasets.fashion_mnist
    (train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
    class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
                   'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

    train_images.shape
    train_images = train_images / 255.0
    test_images = test_images / 255.0

    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(config.layer1_size, activation=config.layer1_activation),
        tf.keras.layers.Dropout(config.dropout_rate),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])

    if config.optimizer == 'rmsprop':
      opt = tf.keras.optimizers.RMSprop(learning_rate = config.learning_rate)
    else:
      opt = tf.keras.optimizers.Adam(learning_rate = config.learning_rate)

    model.compile(optimizer=opt,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    model.fit(train_images, train_labels, epochs=5,
                  validation_data=(test_images, test_labels))


In [14]:
sweep_config = {
    'method': 'grid',
    'parameters': {
        'layer1_size': {
            'values': [64, 96, 128]
        },
        'layer1_activation': {
            'values': ['relu', 'sigmoid']
        },
        'dropout_rate': {
            'values': [0.1, 0.2, 0.3, 0.4, 0.5]
        },
        'optimizer': {
            'values': ['adam', 'rmsprop']
        },
        'learning_rate': {
            'values': [0.1, 0.01, 0.001]
        }
    }
}

In [15]:
import wandb
sweep_id = wandb.sweep(sweep_config, project = 'sweep-practice')

Create sweep with ID: tp4ma51r
Sweep URL: https://wandb.ai/yjchoi/sweep-practice/sweeps/tp4ma51r


In [ ]:
wandb.agent(sweep_id, function=train)

In [25]:
with open("train.py", "w") as f:
  f.write("""
import numpy as np
import tensorflow as tf
import wandb
config_defaults = {
    'layer1_size': 128,
    'layer1_activation':'relu',
    'dropout_rate':0.2,
    'optimizer':'adam',
    'learning_rate':0.01
}
wandb.init(project='sweep-practice', config=config_defaults, magic=True)
config = wandb.config

fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
                'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

train_images.shape
train_images = train_images / 255.0
test_images = test_images / 255.0

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(config.layer1_size, activation=config.layer1_activation),
    tf.keras.layers.Dropout(config.dropout_rate),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

if config.optimizer == 'rmsprop':
  opt = tf.keras.optimizers.RMSprop(learning_rate = config.learning_rate)
else:
  opt = tf.keras.optimizers.Adam(learning_rate = config.learning_rate)

model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5,
              validation_data=(test_images, test_labels))

  """)

In [26]:
!head train.py


import numpy as np
import tensorflow as tf
import wandb
config_defaults = {
    'layer1_size': 128,
    'layer1_activation':'relu',
    'dropout_rate':0.2,
    'optimizer':'adam',
    'learning_rate':0.01


In [ ]:
!wandb agent yjchoi/sweep-practice/tvy4d0c4

wandb: Starting wandb agent 🕵️
2022-02-12 02:32:03,303 - wandb.wandb_agent - INFO - Running runs: []
2022-02-12 02:32:03,544 - wandb.wandb_agent - INFO - Agent received command: run
2022-02-12 02:32:03,545 - wandb.wandb_agent - INFO - Agent starting run with config:
	dropout_rate: 0.11331799786596178
	layer1_activation: relu
	layer1_size: 145
	learning_rate: 0.09776879580750636
	optimizer: rmsprop
2022-02-12 02:32:03,546 - wandb.wandb_agent - INFO - About to run command: /usr/bin/env python train.py --dropout_rate=0.11331799786596178 --layer1_activation=relu --layer1_size=145 --learning_rate=0.09776879580750636 --optimizer=rmsprop
wandb: Currently logged in as: yjchoi (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
2022-02-12 02:32:08,558 - wandb.wandb_agent - INFO - Running runs: ['ue66op2x']
wandb: Tracking run with wandb version 0.12.10
wandb: Syncing run warm-sweep-7
wandb: ⭐️ View project at https://wandb.ai/yjch